# Importer les librairies et les classes nécessaires

In [2]:
import praw
import urllib, urllib.request
import xmltodict
import datetime
import pickle
from Classes import Document, RedditDocument, ArxivDocument
from Classes import Document
from Classes import Author
from Corpus import Corpus

# Fonction pour afficher la structure hiérarchique d'un dictionnaire

In [3]:
def showDictStruct(d):
    def recursivePrint(d, i):
        for k in d:
            if isinstance(d[k], dict):
                print("-"*i, k)
                recursivePrint(d[k], i+2)
            else:
                print("-"*i, k, ":", d[k])
    recursivePrint(d, 1)

# Configuration de l'API Reddit à l'aide la bibliothèque praw

In [4]:
reddit = praw.Reddit(client_id='wfrwVmRrikEa9yhkXWMzMQ', client_secret='ahqeIx-Xp4pbFgGccGsLrrd1aecWkg', user_agent='projet_python')

## Récupération des 100 publications populaires sur Reddit

In [5]:
limit = 100
hot_posts = reddit.subreddit('all').hot(limit=limit)

### Récupération des données à partir de ces 100 publications populaires

In [11]:
docs = []
docs_bruts = []
afficher_cles = False
for i, post in enumerate(hot_posts):
    if i%10==0: print("Reddit:", i, "/", limit)
    # Pour connaître les différentes variables et leur contenu
    if afficher_cles:  
        for k, v in post.__dict__.items():
            pass
            print(k, ":", v)

    # On ne considère pas les posts sans texte
    if post.selftext != "":  
        pass
    docs.append(post.selftext.replace("\n", " "))
    docs_bruts.append(("Reddit", post))

print (docs)

[]


# Récupération des résumés de publications ArXiv avec une requête API


In [8]:
# Paramètres
query_terms = ["clustering", "Dirichlet"]
max_results = 50

# Requête
url = f'http://export.arxiv.org/api/query?search_query=all:{"+".join(query_terms)}&start=0&max_results={max_results}'
data = urllib.request.urlopen(url)

# Format dict (OrderedDict)
data = xmltodict.parse(data.read().decode('utf-8'))

#showDictStruct(data)

# Ajout résumés à la liste
for i, entry in enumerate(data["feed"]["entry"]):
    if i%10==0: print("ArXiv:", i, "/", limit)
    docs.append(entry["summary"].replace("\n", ""))
    docs_bruts.append(("ArXiv", entry))
    #showDictStruct(entry)

ArXiv: 0 / 100
ArXiv: 10 / 100
ArXiv: 20 / 100
ArXiv: 30 / 100
ArXiv: 40 / 100


# Prétraitement et exploitation des données de Reddit et Arxiv

In [9]:
# Affichage et suppression des doublons
print(f"# docs avec doublons : {len(docs)}")
docs = list(set(docs))
print(f"# docs sans doublons : {len(docs)}")

for i, doc in enumerate(docs):
    print(f"Document {i}\t# caractères : {len(doc)}\t# mots : {len(doc.split(' '))}\t# phrases : {len(doc.split('.'))}")
    if len(doc)<100:
        docs.remove(doc)

# Concaténation des documents restants
longueChaineDeCaracteres = " ".join(docs)

# docs avec doublons : 50
# docs sans doublons : 50
Document 0	# caractères : 1096	# mots : 141	# phrases : 8
Document 1	# caractères : 1023	# mots : 132	# phrases : 6
Document 2	# caractères : 1297	# mots : 166	# phrases : 12
Document 3	# caractères : 1360	# mots : 176	# phrases : 10
Document 4	# caractères : 394	# mots : 55	# phrases : 8
Document 5	# caractères : 513	# mots : 63	# phrases : 5
Document 6	# caractères : 1715	# mots : 214	# phrases : 9
Document 7	# caractères : 1012	# mots : 146	# phrases : 7
Document 8	# caractères : 950	# mots : 133	# phrases : 7
Document 9	# caractères : 1006	# mots : 134	# phrases : 7
Document 10	# caractères : 1096	# mots : 157	# phrases : 7
Document 11	# caractères : 756	# mots : 100	# phrases : 8
Document 12	# caractères : 969	# mots : 124	# phrases : 6
Document 13	# caractères : 1831	# mots : 211	# phrases : 12
Document 14	# caractères : 909	# mots : 125	# phrases : 6
Document 15	# caractères : 1876	# mots : 257	# phrases : 11
Document 16	# cara

# Factory Pattern pour la création d'instances de documents à partir de données brutes


In [4]:
collection = []
# FACTORY PATTERN
class DocumentFactory:
    @staticmethod
    def create_document(nature, doc):
        if nature == "ArXiv":  # Les fichiers de ArXiv ou de Reddit ne sont pas formatés de la même manière à ce stade.
        #showDictStruct(doc)

            titre = doc["title"].replace('\n', '')  # On enlève les retours à la ligne
            """try:
                authors = ", ".join([a["name"] for a in doc["author"]])  # On fait une liste d'auteurs, séparés par une virgule
            except:
                authors = doc["author"]["name"]  # Si l'auteur est seul, pas besoin de liste"""
            # récupérer l'auteur et les co-auteur
            try:
                authors_list = [a["name"] for a in doc["author"]]
                primary_author = authors_list[0]
                co_authors = authors_list[1:] if len(authors_list) > 1 else []
            except TypeError:
                primary_author = doc["author"]["name"]
                co_authors = []
            summary = doc["summary"].replace("\n", "")  # On enlève les retours à la ligne
            date = datetime.datetime.strptime(doc["published"], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y/%m/%d")  # Formatage de la date en année/mois/jour avec librairie datetime

            #return ArxivDocument(titre, authors, date, doc["id"], summary) 
            return ArxivDocument(titre, primary_author, date, doc["id"], summary, co_auteurs=co_authors)# Création du Document
           
        elif nature == "Reddit":
            titre = doc.title.replace("\n", '')
            auteur = str(doc.author)
            date = datetime.datetime.fromtimestamp(doc.created).strftime("%Y/%m/%d")
            url = "https://www.reddit.com/"+doc.permalink
            texte = doc.selftext.replace("\n", "")
            nb_commentaires = doc.num_comments
            return RedditDocument(titre, auteur, date, url, texte, nb_commentaires)
        else:
            raise ValueError("Nature de document non supportée.")

### Utilisation du factory pour créer des objets Document

In [13]:
for nature, doc in docs_bruts:
    doc_classe = DocumentFactory.create_document(nature, doc)
    collection.append(doc_classe)

### Création de l'index de documents

In [14]:
id2doc = {}
for i, doc in enumerate(collection):
    id2doc[i] = doc.titre

### Gestion des Auteurs et création d'une liste d'objets Auteur avec index associé


In [15]:
authors = {}
aut2id = {}
num_auteurs_vus = 0

# Création de la liste et l'index des Auteurs
for doc in collection:
    if doc.auteur not in aut2id:
        num_auteurs_vus += 1
        authors[num_auteurs_vus] = Author(doc.auteur)
        aut2id[doc.auteur] = num_auteurs_vus

    authors[aut2id[doc.auteur]].add(doc.texte)

# Construction du corpus à partir des documents

In [6]:
corpus = Corpus("Mon corpus")

for doc in collection:
    corpus.add(doc)
#corpus.show(tri="abc")
#print(repr(corpus))

# Sauvegarde
# Vérifier si le fichier 'corpus.csv' existe déjà
csv_file_exists = path.exists('corpus.csv')
pickle_file_exists = path.exists('corpus.pkl')

if not pickle_file_exists: 
    import pickle

    # Ouverture d'un fichier, puis écriture avec pickle
    with open("corpus.pkl", "wb") as f:
        pickle.dump(corpus, f)

    # Supression de la variable "corpus"
    del corpus

    # Ouverture du fichier, puis lecture avec pickle
    with open("corpus.pkl", "rb") as f:
        corpus = pickle.load(f)

    # La variable est réapparue
    print("Corpus brut", corpus)

if not csv_file_exists:
    # Création du contenu CSV
    nature=corpus.elements_du_corpus()[0]
    titre=corpus.elements_du_corpus()[1]
    Auteur=corpus.elements_du_corpus()[2]
    Co_Auteurs=corpus.elements_du_corpus()[3]
    nb_commentaires=corpus.elements_du_corpus()[4]
    Date=corpus.elements_du_corpus()[5]
    url=corpus.elements_du_corpus()[6]
    texte=corpus.elements_du_corpus()[7]
    
    # Transformation des données en csv
    df = pd.DataFrame(zip(nature,titre,Auteur,Co_Auteurs,Date,url,texte,nb_commentaires), columns=['Nature','Titre','Auteur','Co_Auteurs','Date','URL','Texte','Nb commentaires'])
    df.to_csv(r'corpus.csv',index=False,sep=';')
    
    print("Creation du CSV OK")

else:
    print("Le corpus et le fichier CSV existent déjà.")

print(corpus)

NameError: name 'path' is not defined